In [1]:
import os
import glob
import parsedatetime
import pandas as pd
import numpy as np
from datetime import datetime
from statistics import mode
wd = "/Volumes/som/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Viral Shah/JDRF TIR/3. Data Collection/Cleaned Final Data/"
cal = parsedatetime.Calendar()

In [2]:
# Results dict for storing data
results = {"id":[],"visit":[],"date":[],"tir":[],"a1c":[]}
# Calculate CGM values, etc. for each person
folders = os.listdir(wd + "Cases_T1D+DR")
folders = [f for f in folders if "DS_Store" not in f]
for fol in folders:
    # Get ID
    subject_id = [int(i) for i in fol.split() if i.isdigit()][0]
    # Find summary and CSV files
    files = os.listdir(wd + "Cases_T1D+DR/" + fol)
    csvs = [f for f in files if ".csv" in f]
    summary = [f for f in files if "summary" in f.lower()][0]
    summary = pd.read_excel(wd + "Cases_T1D+DR/" + fol + "/" + summary)
    # Combine all CSV files
    all_data = []
    for c in csvs:
        vis = c.split("_")[0]
        vis = [int(i) for i in vis if i.isdigit()][0]
        cgm = pd.read_csv(wd + "Cases_T1D+DR/" + fol + "/" + c,low_memory = False)
        # Get timestamp and glucose columns, format
        if "Timestamp (YYYY-MM-DDThh:mm:ss)" in cgm.columns:
            cgm = cgm[["Timestamp (YYYY-MM-DDThh:mm:ss)","Glucose Value (mg/dL)"]]
        elif cgm.shape[1] == 19:
            start = cgm.loc[cgm.iloc[:,2] == "Device Timestamp"].index[0]
            cgm.columns = cgm.iloc[start,:]
            cgm = cgm.iloc[start+1:,:]
            cgm = cgm[["Device Timestamp","Historic Glucose mg/dL"]]
        elif cgm.shape[1] > 40:
            start = cgm.loc[cgm['Unnamed: 2'] == "Sensor"].index[0]
            cgm.columns = cgm.iloc[start+1,:]
            cgm = cgm.iloc[start+2:,:]
            cgm.reset_index(inplace = True)
            cgm["timestamp"] = cgm["Date"] + " " + cgm["Time"]
            cgm = cgm[["timestamp","Sensor Glucose (mg/dL)"]]
        cgm.columns = ["timestamp","glucose"]
        cgm.dropna(inplace = True)
        # Glucose to numeric replace "high" and "low" with limits of detection
        cgm["glucose"].replace("High",400,inplace = True)
        cgm["glucose"].replace("Low",40,inplace = True)
        cgm["glucose"] = pd.to_numeric(cgm["glucose"],errors='coerce')
        cgm["timestamp"] = [t.replace("T", " ") for t in cgm["timestamp"]]
        cgm["timestamp"] = [cal.parse(t)[0] for t in cgm["timestamp"]]
        cgm["timestamp"] = [datetime(*t[:6]) for t in cgm["timestamp"]]
        cgm.dropna(inplace = True)
        # Find sampling interval
        s = abs(mode(cgm["timestamp"].diff()).total_seconds())
        # Re-index and expand to every 5 minutes
        cgm.set_index("timestamp",inplace = True)
        cgm = cgm[~cgm.index.duplicated(keep='first')]
        cgm = cgm.resample(str(s / 60) + "T").ffill()
        # Add to list of DFs
        all_data.append(cgm)
    all_data = pd.concat(all_data)
    all_data.reset_index(inplace = True)
    # Split by visit
    dates = [d for d in summary["Office Visit Date"]]
    dates.append(pd.to_datetime("1-1-2100"))
    dates.append(pd.to_datetime("1-1-1900"))
    dates.sort()
    visits = [r for r in range(1,len(dates))]
    visits.sort(reverse = True)
    all_data["visit"] = pd.cut(all_data["timestamp"],bins = dates,labels = visits)
    # Calculate metrics by visit
    df_list = [d for _, d in all_data.groupby(['visit'])]
    df_list = [df for df in df_list if df.shape[0] > 0]
    for d in df_list:
        v = d["visit"].iloc[0]
        date = summary.loc[summary.iloc[:,1] == v]["Office Visit Date"]
        date = str(date.values[0])[:10]
        a1c = float(summary.loc[summary.iloc[:,1] == v]["A1c"])
        total_r = d["glucose"].notna().sum()
        tir = [g for g in d["glucose"] if g >= 70 and g <= 180]
        # Put in results dict
        results["id"].append(subject_id)
        results["visit"].append(v)
        results["date"].append(date)
        results["tir"].append(round(len(tir) / total_r * 100,2))
        results["a1c"].append(a1c)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [3]:
wd + "Cases_T1D+DR/" + fol + "/" + c

'/Volumes/som/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/Projects/Viral Shah/JDRF TIR/3. Data Collection/Cleaned Final Data/Cases_T1D+DR/Case 074/Visit 2_08-16-2018 to 11-13-2018.csv'

In [4]:
summary.loc[summary.iloc[:,1] == v]["Office Visit Date"]

Series([], Name: Office Visit Date, dtype: datetime64[ns])

In [5]:
summary

,DOB,Visit No.,Office Visit Date,A1c,CGM type at visit,Unnamed: 5
0,1989-11-30,1,2019-07-02,6.4,Dexcom,blank clarity export so PDF used
1,NaT,2,2018-11-13,6.4,Dexcom,NaN
2,NaT,3,2018-04-27,7.2,Dexcom,patient's first visit to BDC


In [6]:
v

4

In [8]:
pd.cut(all_data["timestamp"],bins = dates,labels = visits)

0        4
1        4
2        4
3        4
4        4
        ..
33832    3
33833    3
33834    3
33835    3
33836    3
Name: timestamp, Length: 33837, dtype: category
Categories (4, int64): [4 < 3 < 2 < 1]

In [9]:
dates

[Timestamp('1900-01-01 00:00:00'),
 Timestamp('2018-04-27 00:00:00'),
 Timestamp('2018-11-13 00:00:00'),
 Timestamp('2019-07-02 00:00:00'),
 Timestamp('2100-01-01 00:00:00')]